In [21]:
import cv2   
import numpy as np

def PrepareImg(img, result):

    train_data = []
        # Set the threshold for object detection
    conf_threshold = 0.25

        # Set the threshold for non-maximum suppression
    nms_threshold = 0.4

        # Create a list to store the bounding boxes and class names for the detected objects
    boxes = []
    class_names2 = []
    # Number of detected shapes
    numcircles = 0
    numsquares = 0
    numlines = 0
    numstars = 0
    numtriangles = 0
    info = [None] * 22

    

    scores = []


    # Get the scores and the number of detected shapes
    for i in range(len(result.boxes.cls)):
        scores.append(float(result.boxes.cls[i]))
        if int(result.boxes.cls[i] == 0):
            numcircles +=1
        elif int(result.boxes.cls[i] == 1):
            numsquares+=1
        elif int(result.boxes.cls[i] == 2):
            numlines+=1
        elif int(result.boxes.cls[i] == 3):
            numstars+=1
        elif int(result.boxes.cls[i] == 4):  
            numtriangles+=1




    info[0] = numcircles + numsquares + numlines + numstars + numtriangles

    info[1] = numcircles
    info[2] = numsquares
    info[3] = numlines
    info[4] = numstars
    info[5] = numtriangles
    # crop the image into 16 equal regions

    width, height = img.size
    region_width = width // 4
    region_height = height // 4

    regions_percentage = []
# Loop over the regions and extract each one as a separate image
    for i in range(4):
        for j in range(4):
        # Calculate the coordinates of the region
            left = j * region_width
            upper = i * region_height
            right = left + region_width
            lower = upper + region_height
        
            # Crop the image and save it as a separate file
            region = img.crop((left, upper, right, lower))
            region = np.array(region)

            white_pixels = np.sum(region >=127)
            per = (white_pixels / 4096)
            regions_percentage.append(per)
            

    for i in range(0,len(regions_percentage)):
        info[i+6] = regions_percentage[i]        
    

    info = np.array(info, dtype=np.float32)
    
    return info

 
                    
        





In [24]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import keras 
import tensorflow as tf
from PIL import Image

class_names = ['bicycle', 'house' ,'laptop', 'lollipop', 'traffic_light']


img = cv2.imread('Untitled.PNG', cv2.IMREAD_GRAYSCALE)


img = cv2.resize(img, (256,256))
img = np.array(img)
img = img.reshape(-1, 256, 256, 1).astype('float32')
img /=255.0
imgy = Image.fromarray(np.uint8(img.reshape((256, 256))*255))

print(img.shape)
Yolo_Model = YOLO(r'YoloComposedColab100\weights\best.pt')
model = keras.models.load_model(r"SavedModels\NNModelArray.h5")



result = Yolo_Model.predict(source=imgy, save=False,  conf=0.25, task='detect')[0]

predarr = PrepareImg(imgy, result)

predarr = np.array(predarr)
predarr = predarr.reshape(1, -1)  # Reshape predarr to a 2D array with a single row

prediction = model.predict(predarr)[0]
ind = (-prediction).argsort()[:5]
latex = [class_names[x] for x in ind]
print(latex)


prediction = model.predict(predarr)[0]
ind = (-prediction).argsort()[:5]
latex = [class_names[x] for x in ind]
print(latex[0])



(1, 256, 256, 1)



0: 640x640 3 circles, 139.0ms
Speed: 5.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 0s 138ms/step
['traffic_light', 'lollipop', 'bicycle', 'laptop', 'house']
1/1 [==============================] - 0s 73ms/step
traffic_light
